In [57]:
# Import the Dependecies

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [8]:
# Load the data

df = pd.read_csv("train.csv")
df_t = pd.read_csv("test.csv")

X = df.drop("SalePrice", axis=1)
y = df["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1, random_state=1234) 

In [78]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [76]:
num_col = np.array(X.select_dtypes(include=[np.number]).columns)
cat_col = np.array(X.select_dtypes(include=['object']).columns)

43

In [53]:
class PrepProcessor():
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        """ 
        Fill in NaN values in the dataset
        """
        num_col = df.select_dtypes(include=[np.number]).columns
        cat_col = df.select_dtypes(include=['object']).columns
        
        # Transform numeric columns to the mean and categoric columns to "U"
        df[num_col] = df[num_col].fillna(df[num_col].mean())
        df[cat_col] = df[cat_col].fillna("U")
        
        # Drop some Features
#         df.drop(["Id"], axis=1)
        
        return X
    
        

In [54]:
# Build the Pipeline

preprocess = PrepProcessor()
num_pipe = Pipeline([('Scaler', StandardScaler())])
cat_pipe = Pipeline([('OneHot', OneHotEncoder(handle_unknown='ignore'))])
transformer = ColumnTransformer([('num', num_pipe, num_col), ('cat', cat_pipe, cat_col)])

In [60]:
pipe = Pipeline([('InitialPreprocess', preprocess), ('Transformer', transformer), ('xgb', XGBClassifier())])

In [61]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('InitialPreprocess',
                 <__main__.PrepProcessor object at 0x00000299BFEA78E0>),
                ('Transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Scaler',
                                                                   StandardScaler())]),
                                                  array(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'Total...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               objective='multi:softprob', predictor=None, ...))])

In [75]:
yhat = pipe.predict(X_test)
# print(yhat)
# print(y_test)
precision_score(y_test, yhat, average='micro')

0.0